<a href="https://colab.research.google.com/github/peteray-dev/AML_Car_Price_Prediction/blob/master/Sentiment_Analysis_of_Financial_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q cohere praw newsapi-python pandas transformers peft torch huggingface_hub pytz datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datetime import datetime, timedelta
import praw
import cohere
from newsapi import NewsApiClient
import pandas as pd
import logging
import pytz
import time
import warnings
from transformers import LlamaForCausalLM, LlamaTokenizerFast
from peft import PeftModel
import torch
from collections import defaultdict
import numpy as np
import random
from huggingface_hub import login

warnings.filterwarnings("ignore")

In [3]:
from google.colab import userdata

reddit_client_id = userdata.get('REDDIT_CLIENT_ID')
reddit_client_secret = userdata.get('REDDIT_CLIENT_SECRET')
reddit_user_agent = userdata.get('REDDIT_USER_AGENT')
newsapi_key = userdata.get('NEWSAPI_KEY')
cohere_api_key = userdata.get('COHERE_KEY')
huggingface_api_key = userdata.get('HF_TOKEN')
tickers = ["AAPL", "TSLA"]

In [4]:
#
# print(huggingface_api_key)

In [5]:
login(huggingface_api_key)

In [ ]:
model = LlamaForCausalLM.from_pretrained("FinGPT/fingpt-mt_llama3-8b_lora")
tokenizer = LlamaTokenizerFast.from_pretrained("meta-llama/Meta-Llama-3-8B")

adapter_config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
class TextFetchPipeline:
  def __init__(self, newsapi_key, reddit_client_id, reddit_client_secret, reddit_user_agent, tickers, model, tokenizer):
    self.news_api = NewsApiClient(api_key=newsapi_key)
    self.reddit = praw.Reddit(reddit_client_id = reddit_client_id, reddit_client_secret = reddit_client_secret, reddit_user_agent = reddit_user_agent)
    self.co = cohere.Client(cohere_api_key)
    self.model = model
    self.tokenizer = tokenizer
    self.news_cache = set()
    self.reddit_cache = set()
    self.agg_text = {}
    self.sentiment = defaultdict()
    self.tickers = defaultdict()
    self.prop = defaultdict()

    self.ticker_company_map = {"AAPL": "Apple Inc.", "TSLA": "Tesla, Inc."}  # Add more as needed

    for ticker in tickers:
      self.tickers[ticker] = self.ticker_company_map[ticker]
      self.sentiment[ticker] = 'Neutral'
      self.prop[ticker] = 1.0

  def fetch_news(self, ticker):
    company_name = self.tickers[ticker]
    now = datetime.utcnow()
    thirty_minutes_ago = now - timedelta(minutes=30)
    now_str = now.strftime('%Y-%m-%dT%H:%M:%SZ')
    thirty_minutes_ago_str = thirty_minutes_ago.strftime('%Y-%m-%dT%H:%M:%SZ')
    articles = self.news_api.get_everything(q=f"{ticker} OR {company_name}", from_param=thirty_minutes_ago_str, to=now_str, language='en', sort_by='publishAt' ) #relevancy
    new_articles = []
    for article in articles.get["articles", []]:
      title = article['title']
      published_at = datetime.strptime(article["publishedAt"], '%Y-%m-%dT%H:%M:%SZ')
      if title not in self.news_cache and published_at > thirty_minutes_ago:
        self.news_cache.add(title)
        new_articles.append({"source": "news", "text":title, "ticker": ticker})
    return new_articles

  def fetch_reddit(self, ticker):
    company_name = self.tickers[ticker]
    reddit_posts = []
    current_time_utc = datetime.utcnow().replace(tzinfo=pytz.utc)
    self.reddit_cache = {(post_id, ts) for post_id, ts in self.reddit_cache if ts > current_time_utc - timedelta(minutes=10)}
    for post in self.reddit.subreddit('wallstreetbets').new(limit=50):
      text = post.selftext.lower()
      post_time_utc = datetime.utcfromtimestamp(post.created_utc, pytz.UTC)
      if (post.id, post_time_utc) not in self.reddit_cache and (ticker.lower() in text or company_name.lower() in text):
        reddit_posts.append({"source": "reddit", "text": text, "ticker": ticker})
        self.reddit_cache.add((post.id, post_time_utc))
    return reddit_posts

  def fetch_combined_data(self):
    combined_data = []
    for ticker in self.tickers.keys():
      news_data = self.fetch_news(ticker)
      reddit_data = self.fetch_reddit(ticker)
      combined_data.extend(news_data + reddit_data)
      # combined_data.extend(reddit_data)
    return combined_data

  def summarize_text(self, text):
    response = self.co.summarize(text = text, length = "short", format="paragraph", model="summarize-xlarge")
    return response.summary

  def process_combined_data_with_summary(self):
    try:
      est = pytz.timezone('Europe/London')
      raw_data = self.fetch_combined_data()
      if not raw_data:
        for ticker in self.tickers:
          self.agg_text[ticker] = "No data available"
        return

      for entry in raw_data:
        entry["time"] = datetime.utcnow().replace(tzinfo=pytz.utc).astimezone(est).strftime('%Y-%m-%d %H:%M:%S')
      df = pd.DataFrame(raw_data)
      aggregated_df = df.groupby("ticker").agg({'time': "first", "text": lambda texts: " ".join(texts)}).reset_index()
      for ticker in self.tickers:
        filtered_df = aggregated_df[aggregated_df["ticker"] == ticker]
        self.agg_text[ticker] = filtered_df["summary"].values[0] if not filtered_df.empty else ""
    except Exception as e:
      logging.error(f"Error in process_combined_data_with_summary: {str(e)}")

  def get_sentiment(self, text):
    if not text:
      return "Neutral", 1.0
    text = text[:2000]
    prompt = f'''Instruction: What is the sentiment of this news? Please choose an answer from [Positive, Negative, Neutral].\nInput: {text}\nAnswer: '''
    device = torch, device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = self.tokenizer(prompt, return_tensors="pt", padding=True, max_length=128).to(device)
    with torch.no_grad():
      outputs = self.model(**inputs)
    logits = outputs.logits[:, -1, :].to("cpu")
    probs = torch.softmax(logits, dim=-1)
    class_tokens = self.tokenizer(["Positive", "Negative", "Neutral"], add_special_tokens = False)["input_ids"]
    class_probs = {self.tokenizer.decode(token_id): probs[0, token_id].item() for token_id in class_tokens}
    sentiment = max(class_probs, key=class_probs.get)
    prob = np.round(class_probs[sentiment], 2)
    return sentiment, prob

In [ ]:
pipeline = TextFetchPipeline(newsapi_key, reddit_client_id, reddit_client_secret, reddit_user_agent, cohere_api_key, tickers, model, tokenizer)

pipeline.process_combined_data_with_summary()

In [ ]:
for ticker in tickers:
  summary = pipeline.agg_text[ticker]
  sentiment, prob = pipeline.get_sentiment(summary)
  print(f"Ticker: {ticker}, Sentiment: {sentiment}, Confidence: {prob}")